In [9]:

import pandas as pd
import numpy as np
import os

# Try default name used earlier (Colab example)
DATA_PATH = "/content/household_power_consumption.csv"

# If file not found, show helpful message
if not os.path.exists(DATA_PATH):
    raise FileNotFoundError(f"File not found at {DATA_PATH}. Upload the CSV or change DATA_PATH.")

print("Loading file:", DATA_PATH)
df = pd.read_csv(DATA_PATH, sep=',', low_memory=False, encoding='latin1')  # your file had commas
print("File loaded. Rows:", len(df))
print(df.head())
print(df.columns)

Loading file: /content/household_power_consumption.csv
File loaded. Rows: 260640
   index    Date     Time Global_active_power Global_reactive_power Voltage  \
0      0  1/1/07  0:00:00                2.58                 0.136  241.97   
1      1  1/1/07  0:01:00               2.552                   0.1  241.75   
2      2  1/1/07  0:02:00                2.55                   0.1  241.64   
3      3  1/1/07  0:03:00                2.55                   0.1  241.71   
4      4  1/1/07  0:04:00               2.554                   0.1  241.98   

  Global_intensity Sub_metering_1 Sub_metering_2  Sub_metering_3  
0             10.6              0              0             0.0  
1             10.4              0              0             0.0  
2             10.4              0              0             0.0  
3             10.4              0              0             0.0  
4             10.4              0              0             0.0  
Index(['index', 'Date', 'Time', 'Global_ac

In [10]:

# Show column sample values to confirm delimiter correctness
print("First line sample:")
with open(DATA_PATH, 'r', encoding='latin1') as f:
    for _ in range(3):
        print(f.readline().strip())

# If dataset uses semicolon instead of comma, reload:
if df.shape[1] == 1:
    print("Single column detected — reloading with sep=';'")
    df = pd.read_csv(DATA_PATH, sep=';', low_memory=False, encoding='latin1')

# Show basic info
print("\nData shape:", df.shape)
print(df.dtypes)
print("\nMissing counts (first view):")
print(df.isin(['?', ' ?', 'NaN', 'nan', 'NA', '']).sum())


First line sample:
index,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,1/1/07,0:00:00,2.58,0.136,241.97,10.6,0,0,0.0
1,1/1/07,0:01:00,2.552,0.1,241.75,10.4,0,0,0.0

Data shape: (260640, 10)
index                      int64
Date                      object
Time                      object
Global_active_power       object
Global_reactive_power     object
Voltage                   object
Global_intensity          object
Sub_metering_1            object
Sub_metering_2            object
Sub_metering_3           float64
dtype: object

Missing counts (first view):
index                       0
Date                        0
Time                        0
Global_active_power      3771
Global_reactive_power    3771
Voltage                  3771
Global_intensity         3771
Sub_metering_1           3771
Sub_metering_2           3771
Sub_metering_3              0
dtype: int64


In [11]:

# Replace common non-numeric placeholders with np.nan
df.replace(['?', ' ?', 'NaN', 'nan', 'NA', 'N/A', 'null', 'NULL', ''], np.nan, inplace=True)

# Ensure the expected numeric columns exist
expected_num = ['Global_active_power','Global_reactive_power','Voltage',
                'Global_intensity','Sub_metering_1','Sub_metering_2','Sub_metering_3']
missing_expected = [c for c in expected_num if c not in df.columns]
if missing_expected:
    raise KeyError(f"Expected columns missing: {missing_expected}. Check CSV headers.")

# Convert numeric columns to float (coerce invalid -> NaN)
for c in expected_num:
    df[c] = pd.to_numeric(df[c], errors='coerce')

print("Converted numeric dtypes. Any remaining NaNs per numeric column:")
print(df[expected_num].isnull().sum())


Converted numeric dtypes. Any remaining NaNs per numeric column:
Global_active_power      3771
Global_reactive_power    3771
Voltage                  3771
Global_intensity         3771
Sub_metering_1           3771
Sub_metering_2           3771
Sub_metering_3           3771
dtype: int64


In [12]:
# KNN
from sklearn.impute import KNNImputer

numeric_cols = expected_num.copy()
print("Applying KNN imputer to:", numeric_cols)

imputer = KNNImputer(n_neighbors=5, weights='distance')
df[numeric_cols] = imputer.fit_transform(df[numeric_cols])

print("After imputation, missing counts:")
print(df[numeric_cols].isnull().sum())


Applying KNN imputer to: ['Global_active_power', 'Global_reactive_power', 'Voltage', 'Global_intensity', 'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3']
After imputation, missing counts:
Global_active_power      0
Global_reactive_power    0
Voltage                  0
Global_intensity         0
Sub_metering_1           0
Sub_metering_2           0
Sub_metering_3           0
dtype: int64


In [13]:
# DateTime & time features
# Ensure Date & Time columns exist
if 'Date' not in df.columns or 'Time' not in df.columns:
    raise KeyError("Columns 'Date' and/or 'Time' are missing. Check CSV headers.")

# Combine Date + Time into DateTime (handles mixed 2/4-digit years)
df['DateTime'] = pd.to_datetime(df['Date'].astype(str) + ' ' + df['Time'].astype(str),
                                dayfirst=True, errors='coerce', infer_datetime_format=True)

# Drop rows where DateTime failed (rare)
nan_dates = df['DateTime'].isna().sum()
print("Rows with bad DateTime:", nan_dates)
df.dropna(subset=['DateTime'], inplace=True)

# Extract features
df['Hour'] = df['DateTime'].dt.hour
df['Day'] = df['DateTime'].dt.day
df['Month'] = df['DateTime'].dt.month
df['Weekday'] = df['DateTime'].dt.weekday

print("Time features added. Sample:")
print(df[['DateTime','Hour','Day','Month','Weekday']].head())


/tmp/ipython-input-2391237758.py:7: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['DateTime'] = pd.to_datetime(df['Date'].astype(str) + ' ' + df['Time'].astype(str),
/tmp/ipython-input-2391237758.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['DateTime'] = pd.to_datetime(df['Date'].astype(str) + ' ' + df['Time'].astype(str),


Rows with bad DateTime: 0
Time features added. Sample:
             DateTime  Hour  Day  Month  Weekday
0 2007-01-01 00:00:00     0    1      1        0
1 2007-01-01 00:01:00     0    1      1        0
2 2007-01-01 00:02:00     0    1      1        0
3 2007-01-01 00:03:00     0    1      1        0
4 2007-01-01 00:04:00     0    1      1        0


In [14]:
#  Final cleanup
# Remove any exact duplicate rows
print("Duplicates before:", df.duplicated().sum())
df.drop_duplicates(inplace=True)
print("Duplicates after:", df.duplicated().sum())

# If you want, drop original Date / Time columns now
# df.drop(columns=['Date','Time'], inplace=True)

print("Final dataset shape:", df.shape)


Duplicates before: 0
Duplicates after: 0
Final dataset shape: (260640, 15)


In [15]:
# Prepare features and target
feature_cols = [
    'Global_reactive_power','Voltage','Global_intensity',
    'Sub_metering_1','Sub_metering_2','Sub_metering_3',
    'Hour','Day','Month','Weekday'
]
target_col = 'Global_active_power'

# Safety check
for c in feature_cols + [target_col]:
    if c not in df.columns:
        raise KeyError(f"Column missing: {c}")

X = df[feature_cols].copy()
y = df[target_col].copy()

print("X shape:", X.shape, "y shape:", y.shape)
print("Feature sample:")
print(X.head())
print("Target sample:")
print(y.head())


X shape: (260640, 10) y shape: (260640,)
Feature sample:
   Global_reactive_power  Voltage  Global_intensity  Sub_metering_1  \
0                  0.136   241.97              10.6             0.0   
1                  0.100   241.75              10.4             0.0   
2                  0.100   241.64              10.4             0.0   
3                  0.100   241.71              10.4             0.0   
4                  0.100   241.98              10.4             0.0   

   Sub_metering_2  Sub_metering_3  Hour  Day  Month  Weekday  
0             0.0             0.0     0    1      1        0  
1             0.0             0.0     0    1      1        0  
2             0.0             0.0     0    1      1        0  
3             0.0             0.0     0    1      1        0  
4             0.0             0.0     0    1      1        0  
Target sample:
0    2.580
1    2.552
2    2.550
3    2.550
4    2.554
Name: Global_active_power, dtype: float64


In [8]:
#  Train-test split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("Train shapes:", X_train.shape, y_train.shape)
print("Test shapes:", X_test.shape, y_test.shape)


Train shapes: (208512, 10) (208512,)
Test shapes: (52128, 10) (52128,)


In [16]:
feature_cols = [
    'Global_reactive_power','Voltage','Global_intensity',
    'Sub_metering_1','Sub_metering_2','Sub_metering_3',
    'Hour','Day','Month','Weekday'
]
target_col = 'Global_active_power'

# Safety check
for c in feature_cols + [target_col]:
    if c not in df.columns:
        raise KeyError(f"Column missing: {c}")

X_all = df[feature_cols].values      # shape: (N, 10)
y_all = df[target_col].values       # shape: (N,)

print("X_all shape:", X_all.shape)
print("y_all shape:", y_all.shape)

# 3. Time-based train-test split (same for all models)
test_size = 0.2
split_index = int(len(X_all) * (1 - test_size))

X_train_raw = X_all[:split_index]
y_train_raw = y_all[:split_index]
X_test_raw  = X_all[split_index:]
y_test_raw  = y_all[split_index:]

print("Train_raw:", X_train_raw.shape, y_train_raw.shape)
print("Test_raw:",  X_test_raw.shape,  y_test_raw.shape)

X_all shape: (260640, 10)
y_all shape: (260640,)
Train_raw: (208512, 10) (208512,)
Test_raw: (52128, 10) (52128,)


In [17]:
from sklearn.preprocessing import MinMaxScaler

scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_train_scaled = scaler_X.fit_transform(X_train_raw)
X_test_scaled  = scaler_X.transform(X_test_raw)

y_train_scaled = scaler_y.fit_transform(y_train_raw.reshape(-1, 1)).flatten()
y_test_scaled  = scaler_y.transform(y_test_raw.reshape(-1, 1)).flatten()

print("X_train_scaled:", X_train_scaled.shape)
print("X_test_scaled:",  X_test_scaled.shape)

X_train_scaled: (208512, 10)
X_test_scaled: (52128, 10)


In [18]:
import numpy as np

def create_sequences(X, y, seq_len=30):
    X_seq, y_seq = [], []
    for i in range(len(X) - seq_len):
        X_seq.append(X[i:i+seq_len])
        y_seq.append(y[i+seq_len])   # predict the value AFTER the window
    return np.array(X_seq), np.array(y_seq)

SEQ_LEN = 30  # last 30 minutes → predict next minute

# LSTM data
X_train_lstm, y_train_lstm = create_sequences(X_train_scaled, y_train_scaled, seq_len=SEQ_LEN)
X_test_lstm,  y_test_lstm  = create_sequences(X_test_scaled,  y_test_scaled,  seq_len=SEQ_LEN)

print("LSTM train:", X_train_lstm.shape, y_train_lstm.shape)
print("LSTM test:",  X_test_lstm.shape,  y_test_lstm.shape)

# Aligned tabular data for LR / RF / XGBoost
X_train_tab = X_train_scaled[SEQ_LEN:]   # drop first SEQ_LEN
y_train_tab = y_train_scaled[SEQ_LEN:]

X_test_tab  = X_test_scaled[SEQ_LEN:]
y_test_tab  = y_test_scaled[SEQ_LEN:]

print("Tabular train:", X_train_tab.shape, y_train_tab.shape)
print("Tabular test:",  X_test_tab.shape,  y_test_tab.shape)

LSTM train: (208482, 30, 10) (208482,)
LSTM test: (52098, 30, 10) (52098,)
Tabular train: (208482, 10) (208482,)
Tabular test: (52098, 10) (52098,)


In [19]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

lr_model = LinearRegression()
lr_model.fit(X_train_tab, y_train_tab)

y_pred_lr_scaled = lr_model.predict(X_test_tab)

# Inverse-transform to original target scale
y_pred_lr = scaler_y.inverse_transform(y_pred_lr_scaled.reshape(-1, 1)).flatten()
y_test_true_tab = scaler_y.inverse_transform(y_test_tab.reshape(-1, 1)).flatten()

mse_lr = mean_squared_error(y_test_true_tab, y_pred_lr)
r2_lr  = r2_score(y_test_true_tab, y_pred_lr)

print("Linear Regression MSE:", mse_lr)
print("Linear Regression R2:", r2_lr)

Linear Regression MSE: 0.0019918210690076724
Linear Regression R2: 0.997925579182878


In [22]:
from sklearn.ensemble import RandomForestRegressor

rf_model = RandomForestRegressor(
    n_estimators=200,
    max_depth=None,
    random_state=42,
    n_jobs=-1
)

rf_model.fit(X_train_tab, y_train_tab)

y_pred_rf_scaled = rf_model.predict(X_test_tab)

y_pred_rf = scaler_y.inverse_transform(y_pred_rf_scaled.reshape(-1, 1)).flatten()

mse_rf = mean_squared_error(y_test_true_tab, y_pred_rf)
r2_rf  = r2_score(y_test_true_tab, y_pred_rf)

print("Random Forest MSE:", mse_rf)
print("Random Forest R2:", r2_rf)

Random Forest MSE: 0.0016269306316682426
Random Forest R2: 0.9983056014303396


In [21]:
from xgboost import XGBRegressor

xgb_model = XGBRegressor(
    n_estimators=300,
    max_depth=6,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    objective='reg:squarederror'
)

xgb_model.fit(X_train_tab, y_train_tab)

y_pred_xgb_scaled = xgb_model.predict(X_test_tab)

y_pred_xgb = scaler_y.inverse_transform(y_pred_xgb_scaled.reshape(-1, 1)).flatten()

mse_xgb = mean_squared_error(y_test_true_tab, y_pred_xgb)
r2_xgb  = r2_score(y_test_true_tab, y_pred_xgb)

print("XGBoost MSE:", mse_xgb)
print("XGBoost R2:", r2_xgb)

XGBoost MSE: 0.0014197553801469857
XGBoost R2: 0.9985213681280793


In [23]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

num_features = X_train_lstm.shape[2]

lstm_model = Sequential()
lstm_model.add(LSTM(
    units=64,
    return_sequences=False,               # final output of sequence
    input_shape=(SEQ_LEN, num_features)
))
lstm_model.add(Dropout(0.2))
lstm_model.add(Dense(1))                  # regression output

lstm_model.compile(
    loss='mse',
    optimizer=Adam(learning_rate=0.001)
)

lstm_model.summary()

history = lstm_model.fit(
    X_train_lstm, y_train_lstm,
    epochs=20,
    batch_size=32,
    validation_split=0.1,
    shuffle=False,                        # important for time series
    verbose=1
)

# Evaluate
y_pred_lstm_scaled = lstm_model.predict(X_test_lstm).flatten()

y_pred_lstm = scaler_y.inverse_transform(y_pred_lstm_scaled.reshape(-1, 1)).flatten()
y_test_true_lstm = scaler_y.inverse_transform(y_test_lstm.reshape(-1, 1)).flatten()

mse_lstm = mean_squared_error(y_test_true_lstm, y_pred_lstm)
r2_lstm  = r2_score(y_test_true_lstm, y_pred_lstm)

print("LSTM MSE:", mse_lstm)
print("LSTM R2:", r2_lstm)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 64)             │        19,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,265 (75.25 KB)

 Trainable params: 19,265 (75.25 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
5864/5864 ━━━━━━━━━━━━━━━━━━━━ 87s 15ms/step - loss: 0.0039 - val_loss: 7.1959e-04
Epoch 2/20
5864/5864 ━━━━━━━━━━━━━━━━━━━━ 86s 15ms/step - loss: 0.0013 - val_loss: 6.8560e-04
Epoch 3/20
5864/5864 ━━━━━━━━━━━━━━━━━━━━ 85s 15ms/step - loss: 0.0011 - val_loss: 6.8314e-04
Epoch 4/20
5864/5864 ━━━━━━━━━━━━━━━━━━━━ 143s 15ms/step - loss: 0.0010 - val_loss: 6.6279e-04
Epoch 5/20
5864/5864 ━━━━━━━━━━━━━━━━━━━━ 85s 14ms/step - loss: 0.0010 - val_loss: 6.5373e-04
Epoch 6/20
5864/5864 ━━━━━━━━━━━━━━━━━━━━ 85s 15ms/step - loss: 9.8516e-04 - val_loss: 6.4537e-04
Epoch 7/20
5864/5864 ━━━━━━━━━━━━━━━━━━━━ 86s 15ms/step - loss: 9.8012e-04 - val_loss: 6.2771e-04
Epoch 8/20
5864/5864 ━━━━━━━━━━━━━━━━━━━━ 146s 15ms/step - loss: 9.6267e-04 - val_loss: 6.3250e-04
Epoch 9/20
5864/5864 ━━━━━━━━━━━━━━━━━━━━ 86s 15ms/step - loss: 9.4202e-04 - val_loss: 6.2210e-04
Epoch 10/20
5864/5864 ━━━━━━━━━━━━━━━━━━━━ 86s 15ms/step - loss: 9.3668e-04 - val_loss: 5.9921e-04
Epoch 11/20
5864/5864 ━━━━━━━━━━━━━━━

In [26]:
results = {
    "Model": ["Linear Regression", "Random Forest", "XGBoost", "LSTM"],
    "MSE":   [mse_lr, mse_rf, mse_xgb, mse_lstm],
    "R2":    [r2_lr, r2_rf, r2_xgb, r2_lstm]
}

results_df = pd.DataFrame(results)
print(results_df)

               Model       MSE        R2
0  Linear Regression  0.001992  0.997926
1      Random Forest  0.001627  0.998306
2            XGBoost  0.001420  0.998521
3               LSTM  0.079261  0.917452


In [31]:
# ---------- Compare all models and save the best one ----------
import joblib
import tensorflow as tf
from sklearn.metrics import r2_score

# Compute R² scores on test data
scores = {
    'LinearRegression': r2_score(y_test_true_tab, y_pred_lr),
    'RandomForest': r2_score(y_test_true_tab, y_pred_rf),
    'XGBoost': r2_score(y_test_true_tab, y_pred_xgb),
    'LSTM': r2_score(y_test_true_lstm, y_pred_lstm)
}

print("R² scores:", scores)

# Pick the model with the highest R²
best_name = max(scores, key=scores.get)
print("Best model:", best_name)

# Save the best model
if best_name == 'LSTM':
    MODEL_PATH = "/content/best_power_models_lstm.h5"
    best_model = lstm_model
    best_model.save(MODEL_PATH)
    print("Saved best model (LSTM) to:", MODEL_PATH)
else:
    MODEL_PATH = "/content/best_power_models.pkl"
    best_model = {
        'LinearRegression': lr_model,
        'RandomForest': rf_model,
        'XGBoost': xgb_model
    }[best_name]
    joblib.dump(best_model, MODEL_PATH)
    print(f"Saved best model ({best_name}) to:", MODEL_PATH)

R² scores: {'LinearRegression': 0.997925579182878, 'RandomForest': 0.9983056014303396, 'XGBoost': 0.9985213681280793, 'LSTM': 0.9174522293489543}
Best model: XGBoost
Saved best model (XGBoost) to: /content/best_power_models.pkl


In [30]:
# --- DAILY ENERGY CONSUMPTION & ANOMALY DETECTION ---

import pandas as pd

# Ensure Date column is datetime
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)

# Convert Global_active_power (kW per minute) into kWh
df['kWh'] = df['Global_active_power'] * (1/60)

# Group by date to get daily usage
daily_usage = df.groupby(df['Date'].dt.date)['kWh'].sum().reset_index()
daily_usage.rename(columns={'kWh': 'Daily_KWh'}, inplace=True)

print(" Daily Usage Values (First 10 rows):")
display(daily_usage.head(10))

# --------------- Anomaly Detection using IQR -------------------

Q1 = daily_usage['Daily_KWh'].quantile(0.25)
Q3 = daily_usage['Daily_KWh'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

daily_usage['Anomaly'] = daily_usage['Daily_KWh'].apply(
    lambda x: ' Anomaly' if x < lower_bound or x > upper_bound else ' Normal'
)

print("\n IQR Thresholds")
print(f"Lower Limit: {lower_bound:.2f} kWh")
print(f"Upper Limit: {upper_bound:.2f} kWh")

print("\n Daily Usage With Anomaly Tag:")
display(daily_usage)

# Print only anomaly rows
anomalies = daily_usage[daily_usage['Anomaly'] == ' Anomaly']
print("\n Detected Anomaly Days:")
display(anomalies if len(anomalies) > 0 else "No anomalies found")

 Daily Usage Values (First 10 rows):


,Date,Daily_KWh
0,2007-01-01,45.816733
1,2007-01-02,21.153933
2,2007-01-03,16.900900
3,2007-01-04,54.323533
4,2007-01-05,45.222733
5,2007-01-06,25.139633
6,2007-01-07,40.793667
7,2007-01-08,37.356000
8,2007-01-09,31.150900
9,2007-01-10,35.913333



 IQR Thresholds
Lower Limit: -6.73 kWh
Upper Limit: 59.67 kWh

 Daily Usage With Anomaly Tag:


,Date,Daily_KWh,Anomaly
0,2007-01-01,45.816733,Normal
1,2007-01-02,21.153933,Normal
2,2007-01-03,16.900900,Normal
3,2007-01-04,54.323533,Normal
4,2007-01-05,45.222733,Normal
...,...,...,...
176,2007-06-26,15.748467,Normal
177,2007-06-27,24.680433,Normal
178,2007-06-28,21.590633,Normal
179,2007-06-29,21.756049,Normal



 Detected Anomaly Days:


,Date,Daily_KWh,Anomaly
33,2007-02-03,67.162033,Anomaly
34,2007-02-04,59.932333,Anomaly
48,2007-02-18,63.829367,Anomaly
